In [1]:
import os
import sys

module_path = os.path.join(os.getcwd(), '../', 'src')
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from hybrid_search.mapper import DictMapper

file_path = '../data/finance_template_map.xlsx'
sheet_name = 'Income Statement'
dmap = DictMapper(file_path, sheet_name)

target_column = 'C'
base_source_columns = ['F','G']
test_source_columns = ['H','I','J']

base_mapping = dmap.create_mapping_dict(base_source_columns, target_column)
test_mapping = dmap.create_mapping_dict(test_source_columns, target_column)


In [3]:
from hybrid_search.search import HybridSearch
sbert_model_name = "uonyeka/bge-base-financial-matryoshka"
engine = HybridSearch(base_mapping, transformer_model=sbert_model_name)

/home/nunenuh/works/nunenuh/hybrid-search/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /home/nunenuh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nunenuh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/nunenuh/works/nunenuh/hybrid-search/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
%%time
results = engine.hybrid_search("net", top_n=5, bm25_weight=0.5, transformer_weight=0.9)
results


CPU times: user 218 ms, sys: 0 ns, total: 218 ms
Wall time: 33.3 ms


[{'similar_name': 'Net Sales',
  'account_name': 'Revenue',
  'scores': 2.399999976158142},
 {'similar_name': 'Net Revenue',
  'account_name': 'Revenue',
  'scores': 2.123167634010315},
 {'similar_name': 'Other income, net',
  'account_name': 'Other Income',
  'scores': 2.045075237751007},
 {'similar_name': 'Other income/(expense), net',
  'account_name': 'Other Income',
  'scores': 1.5238357186317444},
 {'similar_name': 'Purchases', 'account_name': 'Cost of Sales', 'scores': 0.0}]

In [5]:
%%time
engine.transformer_search("net sales", top_n=5)

CPU times: user 216 ms, sys: 0 ns, total: 216 ms
Wall time: 33.1 ms


[{'similar_name': 'Net Sales', 'account_name': 'Revenue', 'scores': 1.0},
 {'similar_name': 'Net Revenue',
  'account_name': 'Revenue',
  'scores': 0.72147566},
 {'similar_name': 'Other income, net',
  'account_name': 'Other Income',
  'scores': 0.6013825},
 {'similar_name': 'Other income related to sales',
  'account_name': 'Other Operating Income',
  'scores': 0.5863799},
 {'similar_name': 'Cost of Goods Sold',
  'account_name': 'Cost of Sales',
  'scores': 0.53952223}]

In [6]:
%%time
results = engine.bm25_search("Net Sales", top_n=5)
results

CPU times: user 618 μs, sys: 124 μs, total: 742 μs
Wall time: 640 μs


[{'similar_name': 'Net Sales',
  'account_name': 'Revenue',
  'scores': 5.205962434179676},
 {'similar_name': 'Other income related to sales',
  'account_name': 'Other Operating Income',
  'scores': 2.4724117844051094},
 {'similar_name': 'Other income, net',
  'account_name': 'Other Income',
  'scores': 2.247593850770803},
 {'similar_name': 'Net Revenue',
  'account_name': 'Revenue',
  'scores': 2.247593850770803},
 {'similar_name': 'Other income/(expense), net',
  'account_name': 'Other Income',
  'scores': 2.247593850770803}]

In [7]:
from hybrid_search import evaluation
# Evaluate and print the results for each search method
bm25_eval_results, bm25_accuracy = evaluation.evaluate_search_accuracy(test_mapping, engine, engine.bm25_search)
print(f"BM25 Accuracy: {bm25_accuracy:.2f}%")

transformer_eval_results, transformer_accuracy = evaluation.evaluate_search_accuracy(test_mapping, engine, engine.transformer_search)
print(f"Transformer Accuracy: {transformer_accuracy:.2f}%")

hybrid_eval_results, hybrid_accuracy = evaluation.evaluate_search_accuracy(test_mapping, engine, engine.hybrid_search)
print(f"Hybrid Accuracy: {hybrid_accuracy:.2f}%")


BM25 Accuracy: 38.46%
Transformer Accuracy: 53.85%
Hybrid Accuracy: 53.85%


In [8]:
hybrid_eval_results.head()

,Key,Predicted,Ground Truth,Correct,Score
0,Financing revenues,Revenue,Revenue,True,3.7364
1,Producing and manufacturing cost,Cost of Sales,Cost of Sales,True,2.1997
2,"Selling, administrative and general",General and Admin Expenses,Sales and Marketing Expenses,False,0.5473
3,General and administrative,General and Admin Expenses,General and Admin Expenses,True,0.5459
4,employee benefits,General and Admin Expenses,Personnel and Benefit Expenses,False,0.4663


In [9]:
# Generate labels for the hybrid search method
true_labels, hybrid_predicted_labels = evaluation.generate_labels(test_mapping, engine.hybrid_search)

# Print evaluation metrics
evaluation.print_evaluation_metrics(true_labels, hybrid_predicted_labels)

Confusion Matrix Value:
-------------------------
True Positive   : 14
False Positive  : 12
False Negative  : 12
True Negative   : 378


Evaluation Metrics:
--------------------
Precision  0.4375
Recall     0.4271
F1 Score   0.3965
Accuracy   0.5385
